Импорт библиотек:

In [6]:
# ortools and solver
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
from ortools.sat.python import cp_model

# math
import numpy as np
from numpy import sin, cos, tan

Задание основных функций

In [4]:
# Coefficients for stresses calculations

# Buckling coefficient
def K_sm(gamma, beta, phi):
    psi = beta - (np.radians(90) - phi/2)
    m1 = (2 * sin(beta - gamma) / sin(gamma) - 0.7 * tan(1 / (gamma / 2)) - 0.05 / cos(psi))*cos(psi)
    return 1 / m1


# Bending coefficient
def K_iz(gamma, beta, phi):
    psi = beta - (np.radians(90) - phi/2)
    m1 = (2 * sin(beta - gamma) / sin(gamma) - 0.7 * tan(1 / (gamma / 2)) - 0.05 / cos(psi))*cos(psi)
    m4 = (2 * sin(beta - gamma) / sin(gamma) - 0.35 * tan(1 / (gamma / 2)) + 0.35 * tan(1 / (beta / 2))) * sin(gamma) / sin(beta - gamma)
    m5 = m1 / 2 / cos(psi) + 0.35 * tan(1 / (beta / 2)) - m4 / 2 * sin(beta - 90) + 0.05 / cos(psi)
    return 6 * m5 / m4**2 / cos(psi)


# Cut coefficient
def K_sr(gamma, beta, phi):
    psi = beta - (np.radians(90) - phi/2)
    m3 = (2 * sin(beta - gamma) / sin(gamma) - 0.35 * tan(1 / (gamma / 2)) - 0.05 / cos(psi))*sin(gamma) / sin(beta - gamma + phi / 2)
    return 1 / m3

# Fillet radius as function of S (joint step)
def r_func(S):
    return 0.1737 * S - 0.0146 # magic constants are taken by approximation of values from industry standard

# Fastener height
def h_f(z, S, phi): # z -- number of teeth pairs; phi in radians
    return cos(phi / 2) * S * (z - 0.5)


# This function is to find profile width (l_l) of the upper and bottom parts of fastener 
def l_l_first_last(z, S, r, gamma, beta, phi): # z -- number of teeth pairs
    l_d1 = S - 2 * (r / sin(gamma / 2) * cos(gamma / 2 - beta + np.radians(90) - phi / 2) - r) + \
        S * sin(beta - gamma) / sin(gamma) * cos(beta - np.radians(90) + phi / 2) + 8 * sin(phi / 2)
    double_delta = 0.05 * S
    l_l1 = l_d1 - double_delta
    
    l_last = l_l1 - 2 * sin(phi / 2) * S * (z - 0.5)
    return [l_l1, l_last]


# Fir-tree joint approximate volume
def vol_f(l_u, l_b, h, b): # _u -- upper section; _b -- bottom section; 
    S = (l_u + l_b) / 2 * h
    V = S * b
    R_cm = h / 3 * (2 * l_u +l_b) / (l_b + l_u) # http://ru.solverbook.com/question/centr-tyazhesti-trapecii/
    return  [V, R_cm]


# Centrifugal force of blade 
def P_j(vol_u, vol_f, dens, n, R_u, R_f):    # vol_u -- volume of blade above fastener upper section; 
    vol = vol_u + vol_f                      # vol_f -- volume of fastener; dens -- material density; 
    omega = 2 * np.pi * n / 60               # n = rot. speed [rpm]; R_u --  upper blade part center of mass radius;
    dist = R_u - R_f                         # R_f -- fastener center of mass radius
    R = R_f + dist / (1 + vol_f / vol_u)     # dist -- distance between center of masses of blade and fastener
    return vol * dens * omega**2 * R         # R -- the whole blade center of mass


# OST parameter defining width (?)
def l_h(l_l1, S, r, gamma, beta, phi):
    l_h = l_l1 - 2 * (S * sin(beta - gamma) / 2 / sin(gamma) - r / tan(gamma/2)) * \
            cos(beta + phi/2 - np.radians(90)) + 4 * r * sin(beta + phi/2 - np.radians(90)) - 4 * r
    return l_h

Введем переменные

In [7]:
model = cp_model.CpModel()